In [1]:
import pandas as pd
import numpy as np
import sklearn as sklearn
import seaborn as sns
import matplotlib.pyplot as plt
import time

from sklearn import metrics
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import scale
from sklearn.manifold import TSNE

from sklearn import svm
from sklearn import metrics #Import for accuracy calculation

In [2]:
df = pd.read_csv('~/Box/DSCI303/archive/final_data.csv')
df_test = pd.read_csv('~/Box/DSCI303/archive/final_popularity_cut_test.csv')
df_train = pd.read_csv('~/Box/DSCI303/archive/final_popularity_cut_train.csv')
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 152918 entries, 0 to 152917
Data columns (total 26 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   Unnamed: 0              152918 non-null  int64  
 1   Unnamed: 0.1            152918 non-null  int64  
 2   acousticness            152918 non-null  float64
 3   artists                 152918 non-null  object 
 4   danceability            152918 non-null  float64
 5   duration_ms             152918 non-null  int64  
 6   energy                  152918 non-null  float64
 7   explicit                152918 non-null  int64  
 8   id                      152918 non-null  object 
 9   instrumentalness        152918 non-null  float64
 10  key                     152918 non-null  int64  
 11  liveness                152918 non-null  float64
 12  loudness                152918 non-null  float64
 13  mode                    152918 non-null  int64  
 14  name                

In [ ]:
#SVM 
features = ['acousticness', 'danceability', 'energy', 'instrumentalness',
            'liveness', 'loudness','speechiness','tempo','valence']

def doPCA(df, features):
    df_kmeans = df[features]
    pca = PCA(n_components = 3)
    X = pca.fit_transform(df_kmeans)
    print('Explained variation per principal component: {}'.format(pca.explained_variance_ratio_))

    X_reduced = pd.DataFrame(X)
    X_reduced['year_bins'] = df['year_bins']
    X_reduced['popularity_cut'] = df['popularity_cut']
    X_reduced['popularity_qcut'] = df['popularity_qcut']
    
    X_new = pca.inverse_transform(X)
    X_new = pd.DataFrame(X_new, columns=features)
    X_new['year_bins'] = df['year_bins']
    X_new['popularity_cut'] = df['popularity_cut']
    X_new['popularity_qcut'] = df['popularity_qcut']
    return X_reduced, X_new

# def doSVM(popularity_var, kernel='rbf', afterPCA=False):
popularity_var='popularity_cut'
kernel='rbf'
afterPCA=True
df_test = pd.read_csv('~/Box/DSCI303/archive/final_'+popularity_var+'_test.csv')
df_train = pd.read_csv('~/Box/DSCI303/archive/final_'+popularity_var+'_train.csv')
if afterPCA:
    X_reduced, df_test = doPCA(df_test, features)
    X_reduced, df_train = doPCA(df_train, features)
    df_test.info()
X_train = df_train[features]
y_train = df_train[popularity_var]
X_test = df_test[features]
y_test = df_test[popularity_var]
#Create a svm Classifier
time_start = time.time()
clf = svm.SVC(kernel=kernel) # Linear Kernel
#Train the model using the training sets
clf.fit(X_train, y_train)

#Predict the response for test dataset
y_pred = clf.predict(X_test)

# Model Accuracy: how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))
print('SVM done! Time elapsed: {} seconds'.format(time.time()-time_start))
#     return clf, y_pred

# clf_rbf, y_pred_rbf = doSVM('popularity_cut', afterPCA=True)

Explained variation per principal component: [0.34140473 0.15293857 0.1304407 ]
Explained variation per principal component: [0.34375484 0.15097462 0.12720342]
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16991 entries, 0 to 16990
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   acousticness      16991 non-null  float64
 1   danceability      16991 non-null  float64
 2   energy            16991 non-null  float64
 3   instrumentalness  16991 non-null  float64
 4   liveness          16991 non-null  float64
 5   loudness          16991 non-null  float64
 6   speechiness       16991 non-null  float64
 7   tempo             16991 non-null  float64
 8   valence           16991 non-null  float64
 9   year_bins         16991 non-null  int64  
 10  popularity_cut    16991 non-null  int64  
 11  popularity_qcut   16991 non-null  int64  
dtypes: float64(9), int64(3)
memory usage: 1.6 MB


In [ ]:
clf_linear, y_pred_linear = doSVM('popularity_cut','linear', afterPCA=True)

confusion_matrix(y_)